# Clustering Crypto

In [17]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

ImportError: cannot import name 'basestring' from 'holoviews.core.util' (/Users/hillaryhwaga/holoviews/holoviews/core/util.py)

### Fetching Cryptocurrency Data

In [18]:
# Use the following endpoint to fetch json data
url = "https://min-api.cryptocompare.com/data/all/coinlist"

In [19]:
# Create a DataFrame 
# HINT: You will need to use the 'Data' key from the json response, then transpose the DataFrame.
import json
import requests

response = requests.get(url)
Data = json.loads(response.text)

In [20]:
# Alternatively, use the provided csv file:
csv_path = Path("../Data/crypto_data.csv")

# Reading CSV into DataFrame: 
crypto_df = pd.read_csv(csvpath)

# Create a DataFrame
crypto_df.head()

NameError: name 'Path' is not defined

### Data Preprocessing

In [23]:
# Making Unnamed the index

crytpo_df = crypto_df.set_index('Unnamed: 0', inplace=True)

crypto_df.head()

In [ ]:
crypto_df.dtypes

In [ ]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
crypto_df = crypto_df.loc[:,['CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply']]

In [ ]:
# Keep only cryptocurrencies that are trading


In [ ]:
# Keep only cryptocurrencies with a working algorithm


In [ ]:
# Remove the "IsTrading" column
crypto_df = crypto_df.drop("IsTrading",1)

crypto_df.head()

In [ ]:
# Remove rows with at least 1 null value
crypto_df.isnull()

In [ ]:
# pinpointing which is the null
crypto_df.isnull().sum()

In [ ]:
# storing data w/the same df name, then Dropping Null + Remove rows with cryptocurrencies having no coins mined
crypto_df = crypto_df.dropna(subset = ['TotalCoinsMined'])

In [ ]:
# Verifing the Nulls being dropped
crypto_df.isnull().sum()

In [ ]:
crypto_df

In [ ]:
# Drop rows where there are 'N/A' text values
crypto_df = crypto_df.dropna(subset=['Algorithm','ProofType','TotalCoinsMined','TotalCoinSupply'])

In [ ]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
cn_df = crypto_df['CoinName'].copy()
print(cn_df)

In [ ]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
crypto_df = crypto_df.drop(['CoinName'], axis = 1)

In [ ]:
# Create dummy variables for text features
X = pd.get_dummies(data=crypto_df, columns=["Algorithm", "ProofType"])

In [ ]:
X.head()

In [ ]:
# Standardize data
from sklearn.preprocessing import StandardScaler
import numpy as np
X_scaled = StandardScaler().fit_transform(X)
print(X_scaled[0:1])

In [ ]:
X_scaled

### Reducing Dimensions Using PCA

In [ ]:
# Use PCA to reduce dimensions to 3 principal components
from sklearn.decomposition import PCA
pca = PCA(n_components=3)

X_pca = pca.fit_transform(X_scaled)

In [ ]:
# Create a DataFrame with the principal components data
X_pca_df = pd.DataFrame(
    data=X_pca,
    columns = ["Principal Component 1", "Principal Component 2", "Principal Component 3"]
)
X_pca_df.head()

### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [ ]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=1, random_state=0)
    km.fit(X_pca_df)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow =pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

Running K-Means with `k=<your best value for k here>`

In [ ]:
# Initialize the K-Means model
# Initialize the K-Means model
model = KMeans(n_clusters=3, random_state=0)
# Fit the model
model.fit(X_pca_df)
# Predict clusters
predictions = model.predict(X_pca_df)
# Create a new DataFrame including predicted clusters and cryptocurrencies features
print(predictions)

In [ ]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features
new_df = (
    pd.concat([X_pca_df,cn_df], axis=1, sort=True)
    .dropna()
    .reset_index()
)

### Visualizing Results

#### 3D-Scatter with Clusters

In [ ]:
# Create a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(new_df, x="Principal Component 1", y="Principal Component 2", z="Principal Component 3", color="Principal Component 1", symbol="Principal Component 2", hover_name="CoinName", hover_data=['index'], width=800)
fig.show()

#### Table of Tradable Cryptocurrencies

In [ ]:
# Table with tradable cryptos
X_pca_df = X_pca_df.loc{'TotalCoinMined'}

In [ ]:
# Print the total number of tradable cryptocurrencies
X_pca_df.sum()

#### Scatter Plot with Tradable Cryptocurrencies

In [ ]:
# Scale data to create the scatter plot
# Scale data to create the scatter plot
new_df_scaled = new_df
new_df_scaled = StandardScaler.fit_transform(new_df)

In [ ]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
new_df_scaled.hvplot.scatter(
    x="TotalCoinsMined",
    y="TotalCoinSupply",
)